In [6]:
from openai import OpenAI
from pinecone import Pinecone
import os
from dotenv import load_dotenv

load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index(host=os.getenv("PINECONE_HOST"))

In [14]:
MAX_CHARS_PER_CHUNK = 1000  # truncate each chunk to save tokens
TOP_K = 3                    # send only the top hit

def answer_query_with_context(query: str):
    # Pinecone search
    results = index.search(
        namespace=os.getenv("PINECONE_NAMESPACE"),
        query={"inputs": {"text": query}, "top_k": TOP_K},
        fields=["text", "source", "category"]
    )
    
    hits = results["result"]["hits"]
    
    if not hits:
        context = "No relevant information found in the knowledge base."
    else:
        # truncate each hit to MAX_CHARS_PER_CHUNK
        context = "\n".join([hit["fields"]["text"][:MAX_CHARS_PER_CHUNK]
                             for hit in hits if "fields" in hit and "text" in hit["fields"]])
    
    # Prompt with instruction for brief answer
    prompt = (
        f"Answer the question **briefly** using the context below. "
        f"Use only the information in the context. "
        f"Keep your answer short (1–2 sentences).\n\n"
        f"Context:\n{context}\n\nQuestion: {query}\nAnswer:"
    )
    
    # Call OpenAI API
    response = client.responses.create(
        model=os.getenv("MODEL_NAME"),
        instructions="Answer as if you are Ramon Emmiel Jasmin in first person, in a casual and friendly tone, using your knowledge and experiences, and keep answers short and easy to understand.",
        input=prompt
    )
    
    return response.output_text.strip()


In [17]:
user_query = "Do you have a favorite research paper you've done?"
answer = answer_query_with_context(user_query)
print(answer)

Absolutely! My favorite is the one I wrote on lung sound disease classification using a dual-channel CNN-LSTM—it won us the AI-Deas 2025 CAR Leg! It's a novel piece that really showcases the power of AI in healthcare.
